In [1]:
import cv2
recognizer = cv2.face.LBPHFaceRecognizer_create()
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import ggmm.gpu as ggmm

In [2]:
import sys, glob, argparse
import numpy as np
import math
from scipy.stats import multivariate_normal
import time
from sklearn import svm

num_of_images = 10000
# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

def image_vgg_desc(file):
    data = sio.loadmat(file)
    return data['features']

def image_vgg_desc_averaged(file, n):
    data = sio.loadmat(file)
    data = data['features']
    avg_data = [sum(data[0][i:i+n])/n for i in range(0,len(data[0]),n)]
    avg_data = np.array(avg_data).reshape((1, len(avg_data)))
    #print('data shape - ', avg_data.shape)
    return avg_data

def folder_descriptors(folder):
    files = sorted(glob.glob(folder + "/*.mat"))
    files = files[:num_of_images]
    print("Calculating descriptos. Number of images is", len(files))
    return np.concatenate([image_vgg_desc_averaged(file, 4) for file in files])


def generate_gmm(input_folder):
    words = np.concatenate([folder_descriptors(input_folder)]) 
    print("Training GMM of size, input size - ", words.shape)
    
    N, D = words.shape
    K = 32 #128 #256 #4096 #128

    ggmm.init()
    gmm = ggmm.GMM(K,D)

    thresh = 1e-3 # convergence threshold
    n_iter = 20 # maximum number of EM iterations
    init_params = 'wmc' # initialize weights, means, and covariances

    # train GMM
    gmm.fit(words, thresh, n_iter, init_params=init_params)
    
    weights = gmm.get_weights()
    means = gmm.get_means()
    covs = gmm.get_covars()

    print("dict done")

    np.save("means_100.gmm", means)
    np.save("covs_100.gmm", covs)
    np.save("weights_100.gmm", weights)
    print("GMM is done")
    return means, covs, weights

GPU is active


In [3]:


m, c, w = generate_gmm('gen_data/vgg_resize/')

Calculating descriptos. Number of images is 10000
Training GMM of size, input size -  (10000, 1024)
dict done
GMM is done


In [4]:


def likelihood_moment(x, ytk, moment):
    x_moment = np.power(np.float32(x), moment) if moment > 0 else np.float32([1])
    return x_moment * ytk

def likelihood_statistics(samples, means, covs, weights):
    gaussians, s0, s1,s2 = {}, {}, {}, {}
    #print(samples.shape)
    samples = zip(range(0, len(samples)), samples)

    g = [multivariate_normal(mean=means[k], cov=covs[k]) for k in range(0, len(weights)) ]
    for index, x in samples:
        gaussians[index] = np.array([g_k.pdf(x) for g_k in g])

    for k in range(0, len(weights)):
        s0[k], s1[k], s2[k] = 0, 0, 0
        for index, x in samples:
            probabilities = np.multiply(gaussians[index], weights)
            probabilities = probabilities / np.sum(probabilities)
            s0[k] = s0[k] + likelihood_moment(x, probabilities[k], 0)
            s1[k] = s1[k] + likelihood_moment(x, probabilities[k], 1)
            s2[k] = s2[k] + likelihood_moment(x, probabilities[k], 2)

    return s0, s1, s2

def fisher_vector_weights(s0, s1, s2, means, covs, w, T):
    return np.float32([((s0[k] - T * w[k]) / np.sqrt(w[k]) ) for k in range(0, len(w))])

def fisher_vector_means(s0, s1, s2, means, sigma, w, T):
    #print(len(s0), len(s1), len(s2), len(w))
    #print(means[0].shape, sigma[0].shape)
    a = [(s1[k] - means[k][:256] * s0[k]) for k in range(0, len(w))]
    b = [(np.sqrt(w[k] * sigma[0][k])) for k in range(0, len(w))]
    #return np.float32(a/b)
    return np.float32([(s1[k] - means[k][:256] * s0[k]) / (np.sqrt(w[k] * sigma[0][k])) for k in range(0, len(w))])

def fisher_vector_sigma(s0, s1, s2, means, sigma, w, T):
    return np.float32([(s2[k] - 2 * means[k][:256]*s1[k]  + (means[k][:256]*means[k][:256] - sigma[0][k]) * s0[k]) 
                       / (np.sqrt(2*w[k])*sigma[0][k])  for k in range(0, len(w))])

def normalize(fisher_vector):
    v = np.sqrt(abs(fisher_vector)) * np.sign(fisher_vector)
    #print('in normalize')
    return v / np.sqrt(np.dot(v, v))

def fisher_vector(samples, means, covs, w):
    #return 0
    s0, s1, s2 =  likelihood_statistics(samples, means, covs, w)
    T = samples.shape[0]
    #print('in fisher _vector likelihood done')
    #covs = np.float32([np.diagonal(covs[k]) for k in range(0, covs.shape[0])])
    covs = np.float32([np.diagonal(covs)])
    a = fisher_vector_weights(s0, s1, s2, means, covs, w, T)
    b = fisher_vector_means(s0, s1, s2, means, covs, w, T)
    c = fisher_vector_sigma(s0, s1, s2, means, covs, w, T)
    #print('outputs -', a.shape, b[0].shape, c.shape)
    #fv = np.concatenate([np.concatenate(a), np.concatenate(b[0]), np.concatenate(c[0])])
    fv = np.concatenate([a, b[0], c[0]])
    fv = normalize(fv)
    print('image - ', time.time())
    #img_count = img_count + 1
    return fv

"""
def get_fisher_vectors_from_folder(folder, means, covs, w):
    files = glob.glob(folder + "/*.mat")
    files = files[:num_of_images]
    #print (files)
    return np.float32([fisher_vector(image_vgg_desc_averaged(file, 4), means, covs, w) for file in files])
"""
def fisher_features(folder, means, covs, w):
    outpath = 'gen_data/fv_1024/'
    print('means shape - ', means.shape)
    print('covs shape - ', covs.shape)
    print('weights shape - ', w.shape)
    files = sorted(glob.glob(folder + "/*.mat"))
    files = files[:num_of_images]
    
    count = 1
    for file in files:
        fv = np.float32([fisher_vector(image_vgg_desc_averaged(file, 4), means, covs, w)])
        f = file.split('/')
        f = f[2].split('.')
        print("saving the file ", f[0] + '.fv', "count - ", count)
        count = count + 1
        sio.savemat(outpath + f[0] +'.fv', {'fv':fv}, do_compression=True)
    #features = {'vgg_resize' : get_fisher_vectors_from_folder(folder, means, covs, w)}

    return

In [ ]:
t0 = time.time()
fisher_features('gen_data/vgg_resize/', m, c, w)
print('fisher_features are generated')
print("Extracting training fisher vectors for 10000 images took %0.3fs." % (time.time() - t0))

means shape -  (32, 1024)
covs shape -  (32, 1024)
weights shape -  (32,)
image -  1545107237.3748915
saving the file  10.fv count -  1
image -  1545107239.568487
saving the file  100009.fv count -  2
image -  1545107241.7461147
saving the file  100021.fv count -  3
image -  1545107243.8470643
saving the file  100032.fv count -  4
image -  1545107246.0339346
saving the file  100034.fv count -  5
image -  1545107248.2120225
saving the file  100036.fv count -  6
image -  1545107250.4641974
saving the file  100039.fv count -  7
image -  1545107252.8205457
saving the file  10004.fv count -  8
image -  1545107255.2048533
saving the file  100043.fv count -  9
image -  1545107257.558136
saving the file  100048.fv count -  10
image -  1545107259.8534749
saving the file  100061.fv count -  11
image -  1545107262.3600712
saving the file  100073.fv count -  12
image -  1545107264.7359939
saving the file  100087.fv count -  13
image -  1545107267.0686147
saving the file  100088.fv count -  14
imag

image -  1545107507.1033194
saving the file  100701.fv count -  123
image -  1545107509.4135544
saving the file  100718.fv count -  124
image -  1545107511.6932862
saving the file  100721.fv count -  125
image -  1545107513.940449
saving the file  100745.fv count -  126
image -  1545107516.2070696
saving the file  100750.fv count -  127
image -  1545107518.45564
saving the file  100758.fv count -  128
image -  1545107520.7791708
saving the file  100760.fv count -  129
image -  1545107523.0569339
saving the file  100767.fv count -  130
image -  1545107525.292913
saving the file  100771.fv count -  131
image -  1545107527.463744
saving the file  100773.fv count -  132
image -  1545107529.7478046
saving the file  100776.fv count -  133
image -  1545107531.9784818
saving the file  100779.fv count -  134
image -  1545107534.119424
saving the file  100784.fv count -  135
image -  1545107536.400168
saving the file  100790.fv count -  136
image -  1545107538.7403462
saving the file  100800.fv 

image -  1545107777.5337222
saving the file  101385.fv count -  245
image -  1545107779.6882076
saving the file  101392.fv count -  246
image -  1545107781.9261858
saving the file  101411.fv count -  247
image -  1545107784.2002249
saving the file  101420.fv count -  248
image -  1545107786.3069806
saving the file  101423.fv count -  249
image -  1545107788.6407208
saving the file  101429.fv count -  250
image -  1545107790.8257587
saving the file  101432.fv count -  251
image -  1545107793.061688
saving the file  101435.fv count -  252
image -  1545107795.1826925
saving the file  101436.fv count -  253
image -  1545107797.384298
saving the file  101440.fv count -  254
image -  1545107799.655965
saving the file  101444.fv count -  255
image -  1545107801.9899166
saving the file  101447.fv count -  256
image -  1545107804.339861
saving the file  101451.fv count -  257
image -  1545107806.7084537
saving the file  101455.fv count -  258
image -  1545107809.03104
saving the file  101458.fv

image -  1545108050.4127793
saving the file  102130.fv count -  367
image -  1545108052.6243322
saving the file  102131.fv count -  368
image -  1545108054.817254
saving the file  102134.fv count -  369
image -  1545108057.007501
saving the file  102138.fv count -  370
image -  1545108059.2276127
saving the file  102140.fv count -  371
image -  1545108061.4455423
saving the file  102149.fv count -  372
image -  1545108063.700159
saving the file  102151.fv count -  373
image -  1545108065.7843292
saving the file  102154.fv count -  374
image -  1545108067.8914223
saving the file  10216.fv count -  375
image -  1545108070.0243835
saving the file  102170.fv count -  376
image -  1545108072.2373004
saving the file  102171.fv count -  377
image -  1545108074.4882364
saving the file  102176.fv count -  378
image -  1545108076.9066355
saving the file  10218.fv count -  379
image -  1545108079.2391472
saving the file  102180.fv count -  380
image -  1545108081.5101366
saving the file  102187.f

image -  1545108320.36994
saving the file  102845.fv count -  489
image -  1545108322.5717044
saving the file  102849.fv count -  490
image -  1545108324.8292572
saving the file  102853.fv count -  491
image -  1545108327.0432959
saving the file  102858.fv count -  492
image -  1545108329.3693502
saving the file  102863.fv count -  493
image -  1545108331.5491505
saving the file  102870.fv count -  494
image -  1545108333.8500638
saving the file  102871.fv count -  495
image -  1545108336.1074882
saving the file  102883.fv count -  496
image -  1545108338.3664145
saving the file  102886.fv count -  497
image -  1545108340.6350782
saving the file  102888.fv count -  498
image -  1545108342.8630438
saving the file  10289.fv count -  499
image -  1545108345.1489778
saving the file  102896.fv count -  500
image -  1545108347.5579584
saving the file  102904.fv count -  501
image -  1545108349.871878
saving the file  102905.fv count -  502
image -  1545108352.203284
saving the file  102913.f

...

image -  1545116539.6117072
saving the file  123895.fv count -  4216
image -  1545116541.82335
saving the file  123899.fv count -  4217
image -  1545116544.0767622
saving the file  123900.fv count -  4218
image -  1545116546.4839807
saving the file  123901.fv count -  4219
image -  1545116548.8855932
saving the file  123904.fv count -  4220
image -  1545116551.125443
saving the file  123914.fv count -  4221
image -  1545116553.3732605
saving the file  123918.fv count -  4222
image -  1545116555.576514
saving the file  123920.fv count -  4223
image -  1545116557.7769032
saving the file  123928.fv count -  4224
image -  1545116560.1023088
saving the file  123940.fv count -  4225
image -  1545116562.4875016
saving the file  123941.fv count -  4226
image -  1545116564.854302
saving the file  123944.fv count -  4227
image -  1545116567.2492695
saving the file  123953.fv count -  4228
image -  1545116569.5649807
saving the file  123964.fv count -  4229
image -  1545116571.7900052
saving the 

image -  1545116803.3158033
saving the file  124556.fv count -  4336
image -  1545116805.0858314
saving the file  124569.fv count -  4337
image -  1545116806.8497665
saving the file  12457.fv count -  4338
image -  1545116808.6562965
saving the file  12458.fv count -  4339
image -  1545116810.4652805
saving the file  124593.fv count -  4340
image -  1545116812.2461896
saving the file  124595.fv count -  4341
image -  1545116814.152436
saving the file  124603.fv count -  4342
image -  1545116816.469459
saving the file  124604.fv count -  4343
image -  1545116818.684425
saving the file  124606.fv count -  4344
image -  1545116820.9191663
saving the file  12461.fv count -  4345
image -  1545116823.2650428
saving the file  124621.fv count -  4346
image -  1545116825.4690065
saving the file  124622.fv count -  4347
image -  1545116827.6630974
saving the file  124627.fv count -  4348
image -  1545116829.9360635
saving the file  124631.fv count -  4349
image -  1545116832.1373107
saving the f

image -  1545117063.0149865
saving the file  125264.fv count -  4456
image -  1545117065.3607738
saving the file  125272.fv count -  4457
image -  1545117067.703725
saving the file  125277.fv count -  4458
image -  1545117069.9694164
saving the file  125279.fv count -  4459
image -  1545117072.2530296
saving the file  125289.fv count -  4460
image -  1545117074.49847
saving the file  125294.fv count -  4461
image -  1545117076.7012901
saving the file  125304.fv count -  4462
image -  1545117078.948715
saving the file  125314.fv count -  4463
image -  1545117081.278357
saving the file  125321.fv count -  4464
image -  1545117083.6402469
saving the file  125324.fv count -  4465
image -  1545117085.894131
saving the file  125335.fv count -  4466
image -  1545117088.1570256
saving the file  125336.fv count -  4467
image -  1545117090.39316
saving the file  125344.fv count -  4468
image -  1545117092.6852055
saving the file  125347.fv count -  4469
image -  1545117095.0074217
saving the fil

image -  1545117329.7867317
saving the file  126023.fv count -  4576
image -  1545117332.1367962
saving the file  126025.fv count -  4577
image -  1545117334.490953
saving the file  126038.fv count -  4578
image -  1545117336.8074741
saving the file  126047.fv count -  4579
image -  1545117339.1429183
saving the file  126050.fv count -  4580
image -  1545117341.5034919
saving the file  126069.fv count -  4581
image -  1545117343.775532
saving the file  126070.fv count -  4582
image -  1545117345.9963512
saving the file  126072.fv count -  4583
image -  1545117348.2317853
saving the file  126075.fv count -  4584
image -  1545117350.467
saving the file  126077.fv count -  4585
image -  1545117352.702287
saving the file  126080.fv count -  4586
image -  1545117354.8945084
saving the file  126088.fv count -  4587
image -  1545117357.0207045
saving the file  126096.fv count -  4588
image -  1545117359.10226
saving the file  12610.fv count -  4589
image -  1545117361.28223
saving the file  1

image -  1545117592.956895
saving the file  126802.fv count -  4696
image -  1545117595.1506758
saving the file  126809.fv count -  4697
image -  1545117597.3024824
saving the file  126815.fv count -  4698
image -  1545117599.4662776
saving the file  126819.fv count -  4699
image -  1545117601.6730828
saving the file  126822.fv count -  4700
image -  1545117603.9091077
saving the file  126826.fv count -  4701
image -  1545117606.1294925
saving the file  126833.fv count -  4702
image -  1545117608.3543324
saving the file  126837.fv count -  4703
image -  1545117610.6275632
saving the file  126845.fv count -  4704
image -  1545117612.5115385
saving the file  12685.fv count -  4705
image -  1545117614.349531
saving the file  126852.fv count -  4706
image -  1545117616.116917
saving the file  126858.fv count -  4707
image -  1545117617.8865213
saving the file  12686.fv count -  4708
image -  1545117619.66928
saving the file  126888.fv count -  4709
image -  1545117621.4619932
saving the fi